In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
import numpy as np
import pandas as pd

items = pd.read_csv(r"..\Data\Raw\items.csv", delimiter=",")
print(items.head())
print(items.shape)
items = items.drop_duplicates(subset=['item_name'], keep=False)
print(items.shape)

                                           item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edition Full...        1   
2      ***В ЛУЧАХ СЛАВЫ   (UNV)                    D        2   
3    ***ГОЛУБАЯ ВОЛНА  (Univ)                      D        3   
4        ***КОРОБКА (СТЕКЛО)                       D        4   

   item_category_id  
0                40  
1                76  
2                40  
3                40  
4                40  
(22170, 3)
(22170, 3)


In [2]:
test = pd.read_csv(r"..\Data\Raw\test.csv", delimiter=",")
print(test.head())
print(test['shop_id'].nunique())
print(test['ID'].nunique())

   ID  shop_id  item_id
0   0        5     5037
1   1        5     5320
2   2        5     5233
3   3        5     5232
4   4        5     5268
42
214200


In [3]:
sales = pd.read_csv(r"..\Data\Raw\sales_train.csv", delimiter=",")
print(sales.head())
print(sales.shape)
sales = sales.drop_duplicates(subset=['date', 'shop_id', 'item_id'], keep=False)
print(sales.shape)
sales = sales[sales.item_cnt_day<1000]
sales = sales[sales.item_price<60000]
sales = sales[sales.item_price>0]
print(sales.shape)
sales['revenue'] = sales['item_cnt_day'] * sales['item_price']
sales_monthly = sales.groupby(["date_block_num", "shop_id", "item_id"]).agg({'item_cnt_day': ['sum'], 'item_price':['median'], 'revenue':['sum']})
print(sales_monthly.head(20))
sales_monthly.sample(10)

         date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00           1.0
2  05.01.2013               0       25     2552      899.00          -1.0
3  06.01.2013               0       25     2554     1709.05           1.0
4  15.01.2013               0       25     2555     1099.00           1.0
(2935849, 6)
(2935793, 6)
(2935789, 6)
                               item_cnt_day item_price revenue
                                        sum     median     sum
date_block_num shop_id item_id                                
0              0       32               6.0      221.0  1326.0
                       33               3.0      347.0  1041.0
                       35               1.0      247.0   247.0
                       43               1.0      221.0   221.0
                       51               2.0      128.5   257.0
             

,,,item_cnt_day,item_price,revenue
,,,sum,median,sum
date_block_num,shop_id,item_id,,,
8,26,19864,1.0,399.0,399.0
16,15,2318,1.0,224.5,224.5
33,31,20726,2.0,1099.0,2198.0
13,17,1104,1.0,199.0,199.0
16,30,950,1.0,2599.0,2599.0
14,53,10958,1.0,149.0,149.0
15,25,12809,1.0,199.0,199.0
0,27,13756,1.0,149.0,149.0


In [4]:
#join to test set to get only necessary IDs
month_range = range(1, 34)
month_df = pd.DataFrame(month_range)

month_df.rename(columns={0 :'date_block_num'}, inplace=True )
print(month_df.head())
month_df['temp'] = 1
test['temp'] = 1
test_monthly = test.merge(month_df, on=['temp'])
test_monthly.drop(['temp'], axis=1, inplace=True)
whole_df = test_monthly.merge(sales_monthly, how='left', on=['date_block_num', 'item_id', 'shop_id'])
whole_df.sample(10)
print(whole_df.shape)

   date_block_num
0               1
1               2
2               3
3               4
4               5
(7068600, 7)


e:\latest pycharm\venv\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [5]:
categories = pd.read_csv(r"..\Data\Raw\item_categories.csv", delimiter=",")
print(categories.head())


        item_category_name  item_category_id
0  PC - Гарнитуры/Наушники                 0
1         Аксессуары - PS2                 1
2         Аксессуары - PS3                 2
3         Аксессуары - PS4                 3
4         Аксессуары - PSP                 4


In [6]:
print(items.shape)
print(categories.shape)
item_cats = items.merge(categories, how='left', on=['item_category_id'])
print(item_cats.shape)
print(item_cats.head())
item_cats.drop(['item_category_name', 'item_name'], axis=1, inplace=True)
print(item_cats.head())
print(item_cats['item_id'].nunique())
print(item_cats.shape)
whole_df = whole_df.merge(item_cats, how='left', on=['item_id'])
whole_df.sample(10)

(22170, 3)
(84, 2)
(22170, 4)
                                           item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edition Full...        1   
2      ***В ЛУЧАХ СЛАВЫ   (UNV)                    D        2   
3    ***ГОЛУБАЯ ВОЛНА  (Univ)                      D        3   
4        ***КОРОБКА (СТЕКЛО)                       D        4   

   item_category_id                    item_category_name  
0                40                            Кино - DVD  
1                76  Программы - Для дома и офиса (Цифра)  
2                40                            Кино - DVD  
3                40                            Кино - DVD  
4                40                            Кино - DVD  
   item_id  item_category_id
0        0                40
1        1                76
2        2                40
3        3                40
4        4                40
22170
(22170, 2)


,ID,shop_id,item_id,date_block_num,"(item_cnt_day, sum)","(item_price, median)","(revenue, sum)",item_category_id
6411462,194286,41,20608,25,1.0,1999.0,1999.0,72
4565770,138356,57,4181,23,21.0,1190.0,24990.0,75
4630367,140314,57,11567,6,NaN,NaN,NaN,40
2247931,68119,24,15806,5,NaN,NaN,NaN,37
2051835,62176,22,4564,28,NaN,NaN,NaN,55
5436698,164748,36,7070,15,NaN,NaN,NaN,30
2583451,78286,15,12066,14,NaN,NaN,NaN,40
4271022,129424,47,8035,31,NaN,NaN,NaN,55
3427762,103871,42,5660,20,7.0,1590.0,11130.0,3
5259175,159368,56,9541,32,NaN,NaN,NaN,55


In [7]:
whole_df.fillna(0, inplace=True)
list(whole_df.columns)
whole_df.rename(columns={('item_cnt_day', 'sum'):'total_count', ('item_price', 'median'): 'median_price', ('revenue', 'sum'):'total_revenue'}, inplace=True )
whole_df['total_count'].clip(0,20)
print(whole_df.sample(10))
whole_df.to_csv('../Data/Processed/pre_normalised_whole_df.csv', index=False)

             ID  shop_id  item_id  date_block_num  total_count  median_price  \
6462229  195825       41    21706               5          0.0           0.0   
3786104  114730       49     6695              15          0.0           0.0   
5962626  180685       38    14500              22          0.0           0.0   
2426500   73530       21     3080              11          0.0           0.0   
4954736  150143       59    20336              18          0.0           0.0   
1262183   38247       12     6333              33          0.0           0.0   
4921652  149140       59    12805              33          0.0           0.0   
990083    30002        7    18023              18          0.0           0.0   
3718809  112691       49    20836               7          0.0           0.0   
6832849  207056       39    19183               2          0.0           0.0   

         total_revenue  item_category_id  
6462229            0.0                40  
3786104            0.0           

In [31]:
whole_df.sort_values(by=['ID', 'date_block_num'])
print(whole_df.head(34))
whole_df = whole_df[whole_df['ID'] == 46360]
whole_df_arr = whole_df.values
print(whole_df_arr.shape)
whole_df_arr = whole_df_arr.reshape(1, 33, 8)
#change back to 214200

            ID  shop_id  item_id  date_block_num  total_count  median_price  \
1529880  46360       31    20949               1          0.0           0.0   
1529881  46360       31    20949               2          0.0           0.0   
1529882  46360       31    20949               3         13.0           5.0   
1529883  46360       31    20949               4        384.0           5.0   
1529884  46360       31    20949               5        763.0           5.0   
1529885  46360       31    20949               6        799.0           5.0   
1529886  46360       31    20949               7        820.0           5.0   
1529887  46360       31    20949               8        950.0           5.0   
1529888  46360       31    20949               9        978.0           5.0   
1529889  46360       31    20949              10        989.0           5.0   
1529890  46360       31    20949              11       1305.0           5.0   
1529891  46360       31    20949              12    

In [32]:
#prepare training and validation sets
train_arr = whole_df_arr[:, :26]
val_arr = whole_df_arr[:, 26:]
print(train_arr.shape)
print(val_arr.shape)
train_arr_first_test = train_arr[:, 20:26]
print(train_arr_first_test.shape)

#normalise data

train_mean = train_arr.mean(axis=0)
train_std = train_arr.std(axis=0)
train_arr = (train_arr - train_mean)/train_std
val_arr = (val_arr - train_mean)/train_std
print(train_arr_first_test)

(1, 26, 8)
(1, 7, 8)
(1, 6, 8)


<ipython-input-32-8d2a1232d7ce>:13: RuntimeWarning: invalid value encountered in true_divide
  train_arr = (train_arr - train_mean)/train_std


ValueError: operands could not be broadcast together with shapes (1,7,8) (26,8) 

In [16]:
#train_arr = list(train_arr)

#train_arr = train_arr[0:3]
train_list = list(train_arr)
first_test_list = list(train_arr_first_test)
#def split_sequences(sequences, n_steps):
#    pass

In [20]:
window_size = 6

def windowed_dataset(array, window_size):
    ds = tf.data.Dataset.from_tensor_slices(array)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda x: x.batch(window_size + 1))
    ds = ds.map(lambda x: (x[:-1], x[-1][4]))
    return ds
#dataset = windowed_dataset(train_arr, window_size, batch_size)
#for d in dataset:
#   print(d)

In [23]:
print(train_arr.shape)
dataset = tf.data.Dataset.from_generator(lambda: train_list, tf.as_dtype(train_list[0].dtype), tf.TensorShape([26,8]))
dataset = dataset.flat_map(lambda x: windowed_dataset(x, window_size))
dataset = dataset.batch(1).prefetch(1)


(1, 26, 8)


In [ ]:
sequences = [np.arange(5)[:, None, None], np.arange(20, 24)[:, None, None]]
for i in sequences:
    print(i.shape)
#for i in train_list:
#    print(i.shape)

In [27]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True, input_shape=(6,8))),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(lr=1e-3, momentum=0.9))
history = model.fit(dataset,epochs=100,verbose=1)

17/20 [========================>.....] - ETA: 0s - loss: 1384765.8750Epoch 1/100

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

20/20 [==============================] - 0s 7ms/step - loss: 397595.0000
Epoch 2/100
20/20 [==============================] - 0s 6ms/step - loss: 287838.5938

In [ ]:
model.predict(first_test_list)
